In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.2f}".format)

In [2]:
df = pd.read_csv("C:\\Users\\Utkarsh\\Downloads\\properties.csv")
print("Initial shape:", df.shape)

Initial shape: (12685, 145)


In [3]:
df.replace(
    ["NA", "N/A", "null", "", "None"],
    np.nan,
    inplace=True
)

In [4]:
drop_initial = [
    "ID",
    "Price (English)",
    "Data Referred From"
]

all_nan_cols = [
    "Possession By",
    "NRI Pref",
    "Land Area / Covered Area",
    "Land Area Unit",
    "Pantry Type"
]

df.drop(
    columns=[c for c in drop_initial + all_nan_cols if c in df.columns],
    inplace=True
)

In [5]:
df.dropna(axis=1, how="all", inplace=True)
print("After dropping all-null columns:", df.shape)

After dropping all-null columns: (12685, 137)


In [6]:
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
df = df[df["Price"].notna()]

In [7]:
df["Carpet Area"] = pd.to_numeric(df["Carpet Area"], errors="coerce")
df["Covered Area"] = pd.to_numeric(df["Covered Area"], errors="coerce")

df["usable_area_sqft"] = df["Carpet Area"].fillna(df["Covered Area"])

df = df[df["usable_area_sqft"].notna()]

In [8]:
df["price_per_sqft"] = df["Price"] / df["usable_area_sqft"]

In [9]:
low = df["price_per_sqft"].quantile(0.05)
high = df["price_per_sqft"].quantile(0.95)

df = df[df["price_per_sqft"].between(low, high)]
print("After outlier removal:", df.shape)

After outlier removal: (11338, 139)


In [10]:
df["Floor No"] = pd.to_numeric(df["Floor No"], errors="coerce")
df["floors"] = pd.to_numeric(df["floors"], errors="coerce")

df["floor_ratio"] = df["Floor No"] / df["floors"]

In [11]:
df["bedroom"] = pd.to_numeric(df["bedroom"], errors="coerce")
df["Bathroom"] = pd.to_numeric(df["Bathroom"], errors="coerce")

df["bedroom"].fillna(df["bedroom"].median(), inplace=True)
df["Bathroom"].fillna(df["Bathroom"].median(), inplace=True)

df["bathroom_ratio"] = df["Bathroom"] / df["bedroom"]

In [12]:
area_price_mean = df.groupby("Area Name")["Price"].mean()
df["area_price_mean"] = df["Area Name"].map(area_price_mean)

city_price_mean = df.groupby("City")["Price"].mean()
df["city_price_mean"] = df["City"].map(city_price_mean)

In [13]:
amenity_cols = [
    col for col in df.columns
    if df[col].dropna().isin([0, 1]).all()
]

df["amenity_count"] = df[amenity_cols].sum(axis=1)

In [14]:
luxury_features = [
    "Private pool ",
    "Helipad",
    "Skydeck",
    "Concierge Services",
    "Private Jaccuzi"
]

luxury_features = [c for c in luxury_features if c in df.columns]
df["luxury_score"] = df[luxury_features].sum(axis=1)

In [15]:
low_cardinality = [
    "Ownership Type",
    "furnished Type",
    "Transaction Type",
    "Luxury Flat",
    "isPrimeLocationProperty"
]

low_cardinality = [c for c in low_cardinality if c in df.columns]

df = pd.get_dummies(df, columns=low_cardinality, drop_first=True)

In [16]:
df.dropna(
    subset=[
        "Price",
        "usable_area_sqft",
        "bedroom",
        "Bathroom"
    ],
    inplace=True
)

print(df.shape)

(11338, 150)


In [17]:
df.to_csv("properties_cleaned.csv", index=False)